tobac example: Tracking isolated convection based on updraft velocity and total condensate
==

This example notebook demonstrates the use of tobac to track isolated deep convective clouds in cloud-resolving model simulation output based on vertical velocity and total condensate mixing ratio.

The simulation results used in this example were performed as part of the ACPC deep convection intercomparison  case study (http://acpcinitiative.org/Docs/ACPC_DCC_Roadmap_171019.pdf) with WRF using the Morrison microphysics scheme. 

The data used in this example is downloaded from "zenodo link" automatically as part of the notebooks (This only has to be done once for all the tobac example notebooks).

**Import libraries:**

In [1]:
# Import libraries:
import xarray
import numpy as np
import pandas as pd
import os
from six.moves import urllib
from glob import glob

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Import tobac itself:
import tobac

In [3]:
#Disable a couple of warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)


**Download and load example data:**   
The actual dowloading is only necessary once for all example notebooks.

In [4]:
data_out='../'

In [6]:
# # Download the data: This only has to be done once for all tobac examples and can take a while
# file_path='https://zenodo.org/record/3195910/files/climate-processes/tobac_example_data-v1.0.1.zip'
# #file_path='http://zenodo..'
# tempfile='temp.zip'
# print('start downloading data')
# request=urllib.request.urlretrieve(file_path,tempfile)
# print('start extracting data')
# shutil.unpack_archive(tempfile,data_out)
# # zf = zipfile.ZipFile(tempfile)
# # zf.extractall(data_out)
# os.remove(tempfile)
# print('data extracted')


**Load Data from downloaded file:**

In [7]:
data_file_W_mid_max = os.path.join(data_out,'*','data','Example_input_midlevelUpdraft.nc')
data_file_W_mid_max = glob(data_file_W_mid_max)[0]
data_file_TWC = os.path.join(data_out,'*','data','Example_input_Condensate.nc')
data_file_TWC = glob(data_file_TWC)[0]

In [8]:
W_mid_max=xarray.open_dataset(data_file_W_mid_max)['w']
TWC=xarray.open_dataset(data_file_TWC)['twc']

In [9]:
# Display information about the two cubes for vertical velocity and total condensate mixing ratio:
display(W_mid_max)
display(TWC)


<xarray.DataArray 'w' (time: 47, south_north: 198, west_east: 198)>
[1842588 values with dtype=float32]
Coordinates:
  * time                (time) datetime64[ns] 2013-06-19T20:05:00 ... 2013-06-19T23:55:00
  * south_north         (south_north) int64 281 282 283 284 ... 475 476 477 478
  * west_east           (west_east) int64 281 282 283 284 ... 475 476 477 478
    bottom_top_stag     int64 ...
    latitude            (south_north, west_east) float32 ...
    longitude           (south_north, west_east) float32 ...
    model_level_number  int64 ...
    x                   (west_east) float64 ...
    y                   (south_north) float64 ...
    x_0                 (west_east) int64 ...
    y_0                 (south_north) int64 ...
Attributes:
    long_name:     w
    units:         m s-1
    cell_methods:  model_level_number: maximum

<xarray.DataArray 'twc' (time: 47, bottom_top: 94, south_north: 198, west_east: 198)>
[173203272 values with dtype=float32]
Coordinates:
  * time                (time) datetime64[ns] 2013-06-19T20:05:00 ... 2013-06-19T23:55:00
  * bottom_top          (bottom_top) int64 0 1 2 3 4 5 6 ... 88 89 90 91 92 93
  * south_north         (south_north) int64 281 282 283 284 ... 475 476 477 478
  * west_east           (west_east) int64 281 282 283 284 ... 475 476 477 478
    latitude            (south_north, west_east) float32 ...
    longitude           (south_north, west_east) float32 ...
    model_level_number  (bottom_top) int64 ...
    x                   (west_east) float64 ...
    y                   (south_north) float64 ...
    x_0                 (west_east) int64 ...
    y_0                 (south_north) int64 ...
Attributes:
    long_name:  TWC
    units:      kg kg-1

In [10]:
#Set up directory to save output and plots:
savedir='Save'
if not os.path.exists(savedir):
    os.makedirs(savedir)
plot_dir="Plot"
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

**Feature detection:**

Perform feature detection based on midlevel maximum vertical velocity and a range of threshold values.

In [11]:
# Determine temporal and spatial sampling of the input data:
dxy,dt=tobac.utils.get_spacings(W_mid_max)

In [12]:
# Keyword arguments for feature detection step:
parameters_features={}
parameters_features['position_threshold']='weighted_diff'
parameters_features['sigma_threshold']=0.5
parameters_features['min_num']=3
parameters_features['min_distance']=0
parameters_features['sigma_threshold']=1
parameters_features['threshold']=[3,5,10] #m/s
parameters_features['n_erosion_threshold']=0
parameters_features['n_min_threshold']=3

In [13]:
# Perform feature detection and save results:
print('start feature detection based on midlevel column maximum vertical velocity')
dxy,dt=tobac.utils.get_spacings(W_mid_max)
Features=tobac.themes.tobac_v1.feature_detection_multithreshold(W_mid_max,dxy,**parameters_features)
print('feature detection performed start saving features')
Features.to_netcdf(os.path.join(savedir,'Features.nc'))
print('features saved')


start feature detection based on midlevel column maximum vertical velocity
     frame  idx      hdim_1      hdim_2  num  threshold_value  feature  \
0        0    3   86.765659   53.350266   10                3        1   
1        0    6  109.581883   65.230958    7                3        2   
2        0    8  116.808699  191.185786    6                3        3   
3        0   12  155.613171   33.200565    5                3        4   
4        0   14   84.857363   37.092481   43                5        5   
..     ...  ...         ...         ...  ...              ...      ...   
624     46   36  144.848990  116.270462   51               10      625   
625     46   38  158.442623  106.995282    6               10      626   
626     46   39  172.189120    3.980561   53               10      627   
627     46   40  186.763110  101.952079   17               10      628   
628     46   41  190.393192  179.765727    4               10      629   

                    time            

**Segmentation:**  
Perform segmentation based on 3D total condensate field to determine cloud volumes associated to identified features:

In [14]:
parameters_segmentation_TWC={}
parameters_segmentation_TWC['method']='watershed'
parameters_segmentation_TWC['threshold']=0.1e-3  # kg/kg mixing ratio

In [15]:
print('Start segmentation based on total water content')
Mask_TWC,Features_TWC=tobac.themes.tobac_v1.segmentation(Features,TWC,dxy,**parameters_segmentation_TWC)
print('segmentation TWC performed, start saving results to files')
Mask_TWC.to_netcdf(os.path.join(savedir,'Mask_Segmentation_TWC.nc'))
Features_TWC.to_netcdf(os.path.join(savedir,'Features_TWC.nc'))
print('segmentation TWC performed and saved')

Start segmentation based on total water content
<xarray.DataArray 'twc' (time: 47, bottom_top: 94, south_north: 198, west_east: 198)>
[173203272 values with dtype=float32]
Coordinates:
  * time                (time) datetime64[ns] 2013-06-19T20:05:00 ... 2013-06-19T23:55:00
  * bottom_top          (bottom_top) int64 0 1 2 3 4 5 6 ... 88 89 90 91 92 93
  * south_north         (south_north) int64 281 282 283 284 ... 475 476 477 478
  * west_east           (west_east) int64 281 282 283 284 ... 475 476 477 478
    latitude            (south_north, west_east) float32 29.62032 ... 30.515106
    longitude           (south_north, west_east) float32 -94.90857 ... -93.863464
    model_level_number  (bottom_top) int64 0 1 2 3 4 5 6 ... 88 89 90 91 92 93
    x                   (west_east) float64 1.408e+05 1.412e+05 ... 2.392e+05
    y                   (south_north) float64 1.408e+05 1.412e+05 ... 2.392e+05
    x_0                 (west_east) int64 281 282 283 284 ... 475 476 477 478
    y_0    

/home/senf/TROPOS/tmp/tobac-dev-2020-07-09/test-env/lib/python3.7/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
/home/senf/TROPOS/tmp/tobac-dev-2020-07-09/test-env/lib/python3.7/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
/home/senf/TROPOS/tmp/tobac-dev-2020-07-09/test-env/lib/python3.7/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, 

/home/senf/TROPOS/tmp/tobac-dev-2020-07-09/test-env/lib/python3.7/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
/home/senf/TROPOS/tmp/tobac-dev-2020-07-09/test-env/lib/python3.7/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
/home/senf/TROPOS/tmp/tobac-dev-2020-07-09/test-env/lib/python3.7/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, 

segmentation TWC performed, start saving results to files
segmentation TWC performed and saved


**Trajectory linking:**  
Detected features are linked into cloud trajectories using the trackpy library (http://soft-matter.github.io/trackpy). This takes the feature positions determined in the feature detection step into account but does not include information on the shape of the identified objects.

In [16]:
# Keyword arguments for linking step:
parameters_linking={}
parameters_linking['method_linking']='predict'
parameters_linking['adaptive_stop']=0.2
parameters_linking['adaptive_step']=0.95
parameters_linking['extrapolate']=0
parameters_linking['order']=1
parameters_linking['subnetwork_size']=100
parameters_linking['memory']=0
parameters_linking['time_cell_min']=5*60
parameters_linking['method_linking']='predict'
parameters_linking['v_max']=10
parameters_linking['d_min']=2000


In [17]:
# Perform linking and save results:
Track=tobac.themes.tobac_v1.linking_trackpy(Features,W_mid_max,dt=dt,dxy=dxy,**parameters_linking)
Track.to_netcdf(os.path.join(savedir,'Track.nc'))


Frame 46: 18 trajectories present.


**Visualisation:**  

In [18]:
# Set extent for maps plotted in the following cells ( in the form [lon_min,lon_max,lat_min,lat_max])
axis_extent=[-95,-93.8,29.5,30.6]        

In [19]:
# Plot map with all individual tracks:
import cartopy.crs as ccrs
fig_map,ax_map=plt.subplots(figsize=(10,10),subplot_kw={'projection': ccrs.PlateCarree()})
ax_map=tobac.plot.map_tracks(Track,axis_extent=axis_extent,axes=ax_map)

In [21]:
# Create animation showing tracked cells with outline of cloud volumes and the midlevel vertical velocity as a background field:
animation_tobac=tobac.plot.animation_mask_field(Track,Features,W_mid_max,Mask_TWC,
                                          axis_extent=axis_extent,#figsize=figsize,orientation_colorbar='horizontal',pad_colorbar=0.2,
                                          vmin=0,vmax=20,extend='both',cmap='Blues',
                                          interval=500,figsize=(10,7),
                                          plot_outline=True,plot_marker=True,marker_track='x',plot_number=True,plot_features=True)

In [22]:
# Display animation:
from IPython.display import HTML, Image, display
HTML(animation_tobac.to_html5_video())

In [23]:
# # Save animation to file 
# savefile_animation=os.path.join(plot_dir,'Animation.mp4')
# animation_tobac.save(savefile_animation,dpi=200)
# print(f'animation saved to {savefile_animation}')

In [24]:
# Updraft lifetimes of tracked cells:
fig_lifetime,ax_lifetime=plt.subplots()
tobac.plot.plot_lifetime_histogram_bar(Track,axes=ax_lifetime,bin_edges=np.arange(0,120,10),density=False,width_bar=8)
ax_lifetime.set_xlabel('lifetime (min)')
ax_lifetime.set_ylabel('counts')
         

Text(0, 0.5, 'counts')